## Data Model

The Chinook data model represents a digital media store, including tables for artists, albums, media tracks, invoices and customers.

### Objectives:
1. Explore the most popular genres in the USA market
2. Employee Sales Performace analysis
3. Sales across difference countries
4. Whether client prefer purchasing individual tracks or an ablum

### Database Schema:

![image](Schema.jpg)

In [26]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [27]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

   sqlite:///chinook(1).db
 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Explore popular genres in USA market.


Find out which genres sell the most tracks in USA, in order to determine which artist's new albumn will be added to the store collection. 
* separate data into customers from usa only.

In [28]:
%%sql
WITH invoice_usa AS
    (
        SELECT il.* 
        FROM invoice i
            INNER JOIN customer c 
            ON i.customer_id=c.customer_id
            INNER JOIN invoice_line il
            ON il.invoice_id=i.invoice_id
        WHERE c.country = 'USA'
    )

SELECT 
    g.name genre, 
    COUNT(i.invoice_id) sales, 
    cast(COUNT(i.invoice_id) as float)/(select count(*) from invoice_usa) percentage
FROM invoice_usa i
    LEFT JOIN track t 
    ON i.track_id=t.track_id
    LEFT JOIN genre g
    ON t.genre_id=g.genre_id
GROUP BY 1 ORDER BY 2
DESC limit 5

   sqlite:///chinook(1).db
 * sqlite:///chinook.db
Done.


genre,sales,percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423


The store should pick artist who produce music under the following generes;
* Rock
* Alternative & Punk
* Metal
* R&B Soul

-------------------------------------------------------------------------------------------------------------------------------

### Employee Sales performance analysis
Each customer gets assigned to a sales support agent in the company. Find total dollar amouny for each support agent and see who is performing better.

In [29]:
%%sql
WITH employee_id AS
(
    SELECT 
        e.*, 
        c.*, 
        i.*
    FROM employee e
         LEFT JOIN customer c
         ON e.employee_id=c.support_rep_id
         LEFT JOIN invoice i
         ON c.customer_id=i.customer_id
)


SELECT 
ei.first_name||' '||ei.last_name name, 
sum(total) sale 
FROM employee_id ei
GROUP BY name 
ORDER BY sale DESC
LIMIT 3

   sqlite:///chinook(1).db
 * sqlite:///chinook.db
Done.


name,sale
Jane Peacock,1731.510000000004
Margaret Park,1584.0000000000032
Steve Johnson,1393.9200000000028


Top employee is Jane Peacock. The difference may be due to  differences in hiring dates.

---------------------------------------------------------------------------------------------------------------------------
### Analyze sales data for customers from each different country. 

For each country, calculate the following;
* total number of customers
* total value of sales
* average value of sales per customer
* average order value

We will create a 'Other' group, if that country only has 1 customer. 

In [30]:
%%sql

WITH country_or_other AS
    (
     SELECT
         CASE
            WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       i.*
     FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id
    ),
    country_sales AS (
        SELECT 
            country,
            count(distinct customer_id) total_customers,
            sum(total) total_sales,
            sum(total)/count(distinct customer_id) average_sale_per_cus,
            sum(total)/count(invoice_id) average_order_value,
            CASE 
                WHEN country ='Other' 
                THEN 1 
                ELSE 0 
                END AS sort
        FROM country_or_other co
        GROUP BY country
        ORDER BY sort ASC,
        3 DESC
                )

SELECT country,
        total_customers,
        total_sales,
        average_sale_per_cus,
        average_order_value
FROM country_sales

   sqlite:///chinook(1).db
 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,average_sale_per_cus,average_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


USA region contributes to most sales.

There is potential in the following countries, as customers are willing to spent more on average:
* Czech Republic
* India

However, it is important to note that the number of customers in these countries are quite low, therefore this sample size is not large enough and additional research may be required. 

----------------------------------------------------------------------------------------------------
### Sales in Album and Individual tracks
The store is setup in a way that allows customer to make purchases in one of the two ways:

1. purchase a whole album
2. purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In [31]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    ), 
    with_album_or_track AS
    (
        SELECT ift.*, 
        CASE WHEN (
                    SELECT t.track_id
                    FROM track t
                    WHERE t.album_id=(
                                      SELECT t2.album_id
                                        FROM track t2
                                        WHERE t2.track_id=ift.first_track_id)
                    EXCEPT
                    
                    SELECT il.track_id
                    FROM invoice_line il
                    WHERE il.invoice_id =ift.invoice_id) IS NULL
                    
                    AND
        
                   (
                    SELECT il.track_id
                    FROM invoice_line il
                    WHERE il.invoice_id =ift.invoice_id
                    
                    EXCEPT
                       
                    SELECT t.track_id
                    FROM track t
                    WHERE t.album_id=(
                                      SELECT t2.album_id
                                        FROM track t2
                                        WHERE t2.track_id=ift.first_track_id)) IS NULL
                    THEN 'yes'
                    ELSE 'no'
                    END AS 'album_purchase'
                    FROM invoice_first_track ift)

SELECT album_purchase,
        count(invoice_id) number_transactions,
        CAST(count(invoice_id) AS FLOAT)/(SELECT COUNT(*) FROM invoice)sales_percentage
        FROM with_album_or_track 
        GROUP BY 1
        

   sqlite:///chinook(1).db
 * sqlite:///chinook.db
Done.


album_purchase,number_transactions,sales_percentage
no,500,0.8143322475570033
yes,114,0.18566775244299674



Individual track purchase accounts for more than 80% of the sales, therefore there is potenial in changing the store's purchasing strategy to only purchase most popular tracks from each album from record companies, instead of purchasing every track from an album. As album sales on accounts for 19%.